In [2]:
import tensorflow as tf 

x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

In [21]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)

42


In [7]:
sess.close()

In [8]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    print(result)

42


In [10]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()
    print(result)

42


In [16]:
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)

sess.close()


42


In [23]:
#ANY NODE I CREAT IS ADDED TO THE DEFAULT GRAPH
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [24]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)
x2.graph is graph


True

In [25]:
x2.graph is tf.get_default_graph()

False

In [26]:
#when evluate anode tf determines the nodes that it is dependent to and compute them first
w = tf.constant(3)
x = w + 2
y = x + 3
z = x + 5 

with tf.Session() as sess:
    print(y.eval())
    print(z.eval())
    
#the above conde evaluates x and w twice 

8
10


In [38]:
# to conpute hao :
#tf evaluatey and z in one gtaph 

with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val, z_val)

8 10


In [3]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m,n = housing.data.shape
housing_data_with_bias = np.c_[np.ones(shape=(m,1)), housing.data]

X = tf.constant(housing_data_with_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1) , dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta=tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)),XT), y)
 
    
with tf.Session() as sess:
    theta_value = theta.eval()
    print(theta_value)

[[-3.68901253e+01]
 [ 4.36643779e-01]
 [ 9.45042260e-03]
 [-1.07117996e-01]
 [ 6.43712580e-01]
 [-3.96291580e-06]
 [-3.78801115e-03]
 [-4.20931637e-01]
 [-4.34006572e-01]]


In [14]:
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d$H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}".format(root_logdir, now)

In [16]:
#Bath gradient descent
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
scaled_housing_data_plus_bias = scale.fit_transform(housing_data_with_bias)

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
with tf.name_scope("loss") as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name="mse")
#optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum = 0.9)
training_op = optimizer.minimize(mse)
init = tf.global_variables_initializer()
saver = tf.train.Saver({"weights":theta})
#saver = tf.train.import_meta_graph("Models/my_model_final.ckpt.meta")#restore the graph
print(error.op.name)

#mse_summary = tf.summary.scalar('MSE', mse)
#file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())


with tf.Session() as sess:
    saver.restore(sess, "Models/my_model_final.ckpt")
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE = ", mse.eval())
        sess.run(training_op)
        
    best_thetsa = theta.eval()
    #save_path = saver.save(sess, "Models/my_model_final.ckpt")
    
#file_writer.close()

loss/sub
Epoch 0 MSE =  9.477054
Epoch 100 MSE =  4.810266
Epoch 200 MSE =  4.8037033
Epoch 300 MSE =  4.803305
Epoch 400 MSE =  4.8032603
Epoch 500 MSE =  4.8032546
Epoch 600 MSE =  4.8032537
Epoch 700 MSE =  4.8032537
Epoch 800 MSE =  4.8032537
Epoch 900 MSE =  4.8032537


In [20]:
def relu(X,threshold):
    with tf.name_scope("relu"):
        if not hasattr(relu, "threshold"):
            relu.threshold = tf.Variable(0.0, name="threshold")
        w_shape = (int(X.get_shape()[1]),1)
        w = tf.Variable(tf.random_normal(w_shape), name = "weights")
        b = tf.Variable(0.0, name = "bias")
        z = tf.add(tf.matmul(X, w), b, name="z")
        return tf.maximum(z, relu.threshold , name = "output")

n_features = 3
threshold = tf.Variable(0.0, name="threshold")
X  = tf.placeholder(tf.float32, shape=(None, n_features), name = "X")
relus = [relu(X, threshold) for i in range(5)]
output = tf.add_n(relus, name="output")